In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import pandas as pd

train_file_path = '/content/drive/My Drive/small-PhoMT/small-train.json'
dev_file_path = '/content/drive/My Drive/small-PhoMT/small-dev.json'
test_file_path = '/content/drive/My Drive/small-PhoMT/small-test.json'

# Mở và đọc file JSON
with open(train_file_path, 'r', encoding='utf-8') as f:
    train_data = pd.read_json(f)

with open(dev_file_path, 'r', encoding='utf-8') as f:
    dev_data = pd.read_json(f)

with open(test_file_path, 'r', encoding='utf-8') as f:
    test_data = pd.read_json(f)

print(f"Số lượng bản ghi trong train data: {len(train_data)}")
print(f"Số lượng bản ghi trong valid data: {len(dev_data)}")
print(f"Số lượng bản ghi trong test data: {len(test_data)}")

Số lượng bản ghi trong train data: 20000
Số lượng bản ghi trong valid data: 2000
Số lượng bản ghi trong test data: 2000


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   english     20000 non-null  object
 1   vietnamese  20000 non-null  object
dtypes: object(2)
memory usage: 312.6+ KB


In [ ]:
!pip install transformers
!pip install torchtext

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter
import pandas as pd
from transformers import AutoTokenizer
import numpy as np
import random
import math
from tqdm.notebook import tqdm

# Đặt seed để đảm bảo kết quả có thể lặp lại
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

/usr/local/lib/python3.12/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [ ]:
tokenizer_vi = AutoTokenizer.from_pretrained('vinai/phobert-base', use_fast=False)

# Sử dụng Tokenizer cơ bản của Hugging Face cho tiếng Anh (Rất ổn định)
tokenizer_en = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=False)

# Sửa đổi quan trọng: Thêm thuộc tính clean-up
def clean_tokenizer_output(tokens, tokenizer):
    """
    Loại bỏ các token đặc biệt của tokenizer (như [CLS], [SEP], <s>, </s>)
    ra khỏi chuỗi output.
    """
    special_tokens = tokenizer.all_special_tokens
    # Lọc các token đặc biệt ra khỏi output
    return [t for t in tokens if t not in special_tokens]

def tokenize_en(text):
    """Tokenizes tiếng Anh bằng BERT Tokenizer và loại bỏ token đặc biệt."""
    tokens = tokenizer_en.tokenize(text.lower())
    return clean_tokenizer_output(tokens, tokenizer_en) # Lọc token đặc biệt của BERT

def tokenize_vi(text):
    """Tokenizes tiếng Việt bằng PhoBERT Tokenizer và loại bỏ token đặc biệt."""
    tokens = tokenizer_vi.tokenize(text.lower())

    # Lọc token đặc biệt của PhoBERT (<s>, </s>)
    cleaned_tokens = clean_tokenizer_output(tokens, tokenizer_vi)

    # Thêm token <sos> và <eos> đã được định nghĩa trong special_symbols
    return ['<sos>'] + cleaned_tokens + ['<eos>']

# Xây dựng Vocab
def yield_tokens(data_iter, language):
    """Hàm yield tokens từ DataFrame"""
    for _, row in data_iter.iterrows():
        if language == 'en':
            yield tokenize_en(row['english'])
        elif language == 'vi':
            yield tokenize_vi(row['vietnamese'])

# Các token đặc biệt
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

# Xây dựng Vocab
vocab_src = build_vocab_from_iterator(
    yield_tokens(train_data, 'en'),
    min_freq=2, # Chỉ giữ lại các từ xuất hiện ít nhất 2 lần
    specials=special_symbols
)
vocab_trg = build_vocab_from_iterator(
    yield_tokens(train_data, 'vi'),
    min_freq=2,
    specials=special_symbols
)

# Tự động gán index UNK_IDX cho những từ nằm ngoài vocab.
vocab_src.set_default_index(UNK_IDX)
vocab_trg.set_default_index(UNK_IDX)

# Kích thước từ vựng
INPUT_DIM = len(vocab_src)
OUTPUT_DIM = len(vocab_trg)
print(f"Kích thước từ vựng tiếng Anh (Input): {INPUT_DIM}")
print(f"Kích thước từ vựng tiếng Việt (Output): {OUTPUT_DIM}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Kích thước từ vựng tiếng Anh (Input): 10699
Kích thước từ vựng tiếng Việt (Output): 4699


In [7]:
# Biến dữ liệu DataFrame thành một bộ sưu tập các cặp Tensor đã được chỉ số hóa, sẵn sàng được nạp vào mô hình.
class TranslationDataset(Dataset):
    def __init__(self, df, vocab_src, vocab_trg):
        self.df = df
        self.vocab_src = vocab_src
        self.vocab_trg = vocab_trg

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        en_text = self.df.iloc[idx]['english']
        vi_text = self.df.iloc[idx]['vietnamese']

        # Tách token
        en_tokens = tokenize_en(en_text)
        vi_tokens = tokenize_vi(vi_text)

        # Chuyển token sang index
        en_indices = self.vocab_src.lookup_indices(en_tokens)
        vi_indices = self.vocab_trg.lookup_indices(vi_tokens)

        return torch.LongTensor(en_indices), torch.LongTensor(vi_indices)

# Collate function để padding các chuỗi trong cùng một batch
def collate_fn(batch):
    src_batch, trg_batch = [], []
    for src, trg in batch:
        src_batch.append(src)
        trg_batch.append(trg)

    # Padding: Hàm pad_sequence sẽ mặc định padding đến độ dài lớn nhất của sample trong batch
    src_batch = nn.utils.rnn.pad_sequence(src_batch, padding_value=PAD_IDX)
    trg_batch = nn.utils.rnn.pad_sequence(trg_batch, padding_value=PAD_IDX)

    return src_batch, trg_batch

# Tạo Dataset và DataLoader
BATCH_SIZE = 128
train_dataset = TranslationDataset(train_data, vocab_src, vocab_trg)
dev_dataset = TranslationDataset(dev_data, vocab_src, vocab_trg)
test_dataset = TranslationDataset(test_data, vocab_src, vocab_trg)

train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
dev_iterator = DataLoader(dev_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        # input_dim: Kích thước từ vựng tiếng Anh (INPUT_DIM)
        # emb_dim: Kích thước embedding (ví dụ: 256)
        # hid_dim: Kích thước ẩn (hidden size): 256
        # n_layers: Số lớp LSTM: 3

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        # Lớp embedding, giúp chuyển đổi thành dense vector
        self.embedding = nn.Embedding(input_dim, emb_dim)

        # 3 lớp LSTM (batch_first=False: Seq_Len x Batch_Size x Features)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)

        # Lớp này dùng để tắt ngẫu nhiên chiều của đầu ra của lớp embedding
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src_len, batch_size]

        embedded = self.dropout(self.embedding(src))
        # embedded = [src_len, batch_size, emb_dim]

        # output: output cho mỗi timestep (thường không dùng)
        # (hidden, cell): final hidden state và cell state
        output, (hidden, cell) = self.rnn(embedded)

        # output = [src_len, batch_size, hid_dim * n_directions]
        # hidden = [n_layers, batch_size, hid_dim]
        # cell = [n_layers, batch_size, hid_dim]

        # Chỉ trả về final hidden và cell state
        return hidden, cell

In [9]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        # output_dim: Kích thước từ vựng tiếng Việt (OUTPUT_DIM)
        # Các tham số khác phải khớp với Encoder

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)

        # 3 lớp LSTM (phải có kích thước và số lớp khớp với Encoder)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)

        # Linear layer để dự đoán từ tiếp theo
        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch_size] -> chỉ là 1 từ tại 1 timestep
        # hidden = [n_layers, batch_size, hid_dim]
        # cell = [n_layers, batch_size, hid_dim]

        # Thêm 1 chiều (sequence length) = 1
        input = input.unsqueeze(0)
        # input = [1, batch_size]

        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch_size, emb_dim]

        # output: output cho timestep hiện tại
        # (hidden, cell): hidden/cell state đã cập nhật
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        # output = [1, batch_size, hid_dim * n_directions]
        # hidden = [n_layers, batch_size, hid_dim]
        # cell = [n_layers, batch_size, hid_dim]

        # Loại bỏ chiều sequence length = 1
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch_size, output_dim] -> logits cho từ tiếp theo

        return prediction, hidden, cell


In [10]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # Đảm bảo hid_dim của encoder và decoder là giống nhau
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        # Đảm bảo n_layers của encoder và decoder là giống nhau
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        # src = [src_len, batch_size]
        # trg = [trg_len, batch_size]

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # Tensor để lưu trữ các dự đoán
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # Trạng thái ẩn ban đầu (Context Vector)
        hidden, cell = self.encoder(src)

        # First input to the decoder là token <sos>
        input = trg[0,:]

        for t in range(1, trg_len):

            # Dự đoán từ tiếp theo
            output, hidden, cell = self.decoder(input, hidden, cell)

            # Lưu dự đoán
            outputs[t] = output

            # Quyết định sử dụng Teacher Forcing hay không
            teacher_force = random.random() < teacher_forcing_ratio

            # Lấy chỉ số của từ có xác suất cao nhất
            top1 = output.argmax(1)

            # Nếu teacher_force, input là từ đúng (target word)
            # Ngược lại, input là từ đã dự đoán
            input = trg[t] if teacher_force else top1

        return outputs

In [11]:
# Kích thước cố định theo yêu cầu bài toán
HID_DIM = 256
N_LAYERS = 3

# Các tham số khác có thể điều chỉnh
EMB_DIM = 256 # Kích thước embedding, có thể bằng HID_DIM
DROPOUT = 0.5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)

model = Seq2Seq(enc, dec, DEVICE).to(DEVICE)

In [12]:
# Khởi tạo trọng số ban đầu hợp lý để tránh vanishing, exploding
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10699, 256)
    (rnn): LSTM(256, 256, num_layers=3, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(4699, 256)
    (rnn): LSTM(256, 256, num_layers=3, dropout=0.5)
    (fc_out): Linear(in_features=256, out_features=4699, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [13]:
# Tối ưu: Adam (theo yêu cầu)
optimizer = optim.Adam(model.parameters())

# Hàm mất mát: CrossEntropyLoss, bỏ qua PAD token
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [14]:
def train(model, iterator, optimizer, criterion, clip):

    model.train()
    epoch_loss = 0

    for i, (src, trg) in enumerate(tqdm(iterator, desc="Training")):

        src = src.to(DEVICE)
        trg = trg.to(DEVICE)

        optimizer.zero_grad()

        # outputs = [trg_len, batch_size, trg_vocab_size]
        output = model(src, trg)

        # trg = [trg_len, batch_size]
        # Cần flatten output và trg để tính Loss
        # Skip token <sos> ở index 0
        output_dim = output.shape[-1]

        # output_flat = [(trg_len - 1) * batch_size, output_dim]
        output = output[1:].view(-1, output_dim)

        # trg_flat = [(trg_len - 1) * batch_size]
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        # Cắt gradient để tránh hiện tượng exploding gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):

    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, (src, trg) in enumerate(tqdm(iterator, desc="Evaluating")):

            src = src.to(DEVICE)
            trg = trg.to(DEVICE)

            # Tắt teacher forcing (ratio = 0)
            output = model(src, trg, 0) #turn off teacher forcing

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [15]:
N_EPOCHS = 20
CLIP = 1
best_dev_loss = float('inf')

for epoch in range(N_EPOCHS):

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    dev_loss = evaluate(model, dev_iterator, criterion)

    if dev_loss < best_dev_loss:
        best_dev_loss = dev_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val. Loss: {dev_loss:.3f} | Val. PPL: {math.exp(dev_loss):.3f}')

Training:   0%|          | 0/157 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 01 | Train Loss: 6.450 | Val. Loss: 6.470 | Val. PPL: 645.517


Training:   0%|          | 0/157 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 19 | Train Loss: 5.344 | Val. Loss: 6.143 | Val. PPL: 465.524


Training:   0%|          | 0/157 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch: 20 | Train Loss: 5.312 | Val. Loss: 6.150 | Val. PPL: 468.931


In [16]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=28b417c62f6db2069bb5b58a232de2e683d339a7d7bcc214abb697a7cb38d477
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer


# Tải mô hình tốt nhất
model.load_state_dict(torch.load('tut1-model.pt'))

def translate_sentence(sentence, src_vocab, trg_vocab, model, device, max_len = 50):
    model.eval()

    # --- SỬA ĐỔI BƯỚC 1: TOKENIZE DÙNG HÀM MỚI (tokenize_en) ---
    # 1. Tokenize và Index

    # 1.1 Tokenize bằng hàm mới (sử dụng BERT Tokenizer)
    tokens = tokenize_en(sentence) # Dùng hàm đã định nghĩa

    # 1.2 Chuyển token thành index
    # lookup_indices nhận một danh sách, nên chúng ta không cần lặp
    src_indexes = src_vocab.lookup_indices(tokens)

    # Thêm chiều batch (batch_size=1)
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 2. Encoding
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    # 3. Decoding
    trg_indexes = [trg_vocab['<sos>']] # Bắt đầu bằng <sos>

    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)

        if pred_token == trg_vocab['<eos>']:
            break

    # 4. Chuyển Index thành Text
    trg_tokens = trg_vocab.lookup_tokens(trg_indexes)

    # Trả về chuỗi token, bỏ <sos> ở đầu và <eos> (hoặc <pad>) ở cuối
    return trg_tokens[1:]

def calculate_rouge_l(model, iterator, src_vocab, trg_vocab, device):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    total_rouge_l = 0
    count = 0

    for src_batch, trg_batch in tqdm(iterator, desc="Calculating ROUGE-L"):
        # Trg (ground truth)
        trg_batch = trg_batch.transpose(0, 1) # [batch_size, trg_len]

        for i in range(trg_batch.shape[0]):
            # Chuyển sentence source tensor sang text
            src_tensor = src_batch[:, i].unsqueeze(1)
            src_indices = src_tensor.squeeze(1).tolist()
            # 1. Loại bỏ <pad> và <unk> khỏi câu nguồn trước khi dịch
            src_tokens_raw = src_vocab.lookup_tokens(src_indices)
            src_tokens = [t for t in src_tokens_raw if t not in ['<pad>', '<unk>']]
            src_sentence = ' '.join(src_tokens)

            # Dịch sentence
            hyp_tokens = translate_sentence(src_sentence, src_vocab, trg_vocab, model, device)
            # 2. Xử lý hypothesis (câu dự đoán): Loại bỏ <unk> và các token thừa
            hypothesis = ' '.join([t for t in hyp_tokens if t not in ['<unk>', '<eos>', '<pad>']]).strip()

            # Ground truth
            ref_tokens_raw = trg_vocab.lookup_tokens(trg_batch[i].tolist())
            # 3. Xử lý reference (câu tham chiếu): Loại bỏ tất cả các token đặc biệt
            reference = ' '.join([t for t in ref_tokens_raw if t not in ['<sos>', '<eos>', '<pad>', '<unk>']]).strip()

            if not reference or not hypothesis:
                continue

            score = scorer.score(reference, hypothesis)
            total_rouge_l += score['rougeL'].fmeasure
            count += 1

    return total_rouge_l / count if count > 0 else 0

rouge_l_score = calculate_rouge_l(model, test_iterator, vocab_src, vocab_trg, DEVICE)
print(f"\n✅ Đánh giá ROUGE-L trên tập Test: {rouge_l_score:.4f}")

Calculating ROUGE-L:   0%|          | 0/16 [00:00<?, ?it/s]


✅ Đánh giá ROUGE-L trên tập Test: 0.2469
